In [21]:
# WEB SCRAPING FOR JOBSTREET

import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_url(position,location):
    position = position.replace(" ","-")
    location = location.replace(" ","-")
    template = 'https://www.jobstreet.com.my/en/job-search/{}-jobs-in-{}'
    url= template.format(position,location)
    return url

def get_jobs(card):
    
    # 1) getting job title : job_title = card.find('div','sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0').text.strip()
    try:
        name = card.find('div','sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0').text.strip()
    
    except AttributeError:
        name = ""
    
    # 2)getting job url
    anchor_tag = card.h1.a
    job_url = 'https://www.jobstreet.com.my'+ anchor_tag.get('href')
    
    
    # 3)fetching company name, using try except
    try:
        company= card.find('span','sx2jih0 zcydq82b _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8').text
        
    except AttributeError:
        company = ""

   # 4) fetching company location, using try except  
    try: 
        job_location = card.find('span','sx2jih0 zcydq82b zcydq8r iwjz4h0').text
    except AttributeError:
        location = ''
        
        
    # 5) fetching job summary
    try:
        summary =''
        tag_str = ''
        job_summary = card.find("ul","sx2jih0 sx2jih3 h6p8rp0 h6p8rp5")
        for tag in job_summary('li'):
            tag_str=tag.text
            summary += tag_str + ','
            summary = summary.rstrip(',')
    except AttributeError:
        summary = ''
    
    # 6) fetching job salary
    job_salary = card.find_all('span',"sx2jih0 zcydq82b _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc6")
    try:
        if (len(job_salary)) >= 2:
            job_salary = job_salary[1]
            salary = job_salary.text.strip()
        else:
            salary= ''
    except IndexError:
        salary=''
    
    time_tag = card.time
    post_date = time_tag.get('datetime') 
    post_date = post_date.split("T",1)
    post_date = post_date[0]
    today = datetime.today().strftime('%Y-%m-%d')
    
    job_info = (name, job_url, company, 
                job_location, summary, salary, post_date, today)
    
    return job_info

def main(position,location):
    records = []
    url = get_url(position,location)
    
    response=requests.get(url)
    
    soup = BeautifulSoup(response.text,'html.parser')
    
    cards = soup.find_all("div","sx2jih0 zcydq83w zcydq835 zcydq85e zcydq84n zcydq81w zcydq82k zcydq89w zcydq89n")
    
    for everyCard in cards:
        jobDetails = get_jobs(everyCard)
        records.append(jobDetails)
        
    # here we are using pandas DataFrame
    
    cols = ["Job_Name","Job_URL", "Company_Name", "Job_Location",
           "Job_Summary","Job_salary","Post_Date", "Today_Date"]
     
    job_street_data = pd.DataFrame(records, columns = cols)
    job_street_data.to_csv("D:\\JobStreetdata.csv", encoding ="utf-8")
    

In [22]:
main("PHP Developer","Kuala Lumpur")